# calcular_tendencias.py

### Con este programa representamos la frecuencia de cada clase, en una región determinada del hemisferio norte que escojamos, con respecto al año y obtenemos la regresión lineal de dicha representación, analizando además el p-valor para ver la significancia estadística

#### Cargar modulos

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import pymannkendall as mk

#### Parametros de entrada

In [3]:
DatosEntrada="C:\\Users\\Usuario\\Desktop\\Practicas Meteogalicia\\Datos de entrada\\wtseries_era5_historical_r1i1p1_nh_1979_2020.nc"
lat_galicia=43
lon_galicia=360-9
labelregion="Galcia"
datos_tipos=['PA', 'DANE', 'DAE', 'DASE', 'DAS', 'DASW', 'DAW', 'DANW', 'DAN', 'PDNE', 'PDE', 'PDSE', 'PDS', 'PDSW', 'PDW', 'PDNW', 'PDN', 'PC', 'DCNE', 'DCE', 'DCSE', 'DCS', 'DCSW', 'DCW', 'DCNW', 'DCN', 'U']

In [4]:
datos=xr.open_dataset(DatosEntrada) #El formato es xarray.core.dataset.Dataset
datos_tiempo=datos.time #El formato es xarray.core.dataarray.DataArray
datos_tiempo=datos_tiempo.to_pandas() #El formato es un pandas.core.series.Series
datos_lon=datos.lon.values
datos_lat=datos.lat.values #Estos dos últimos son arrays

datos_galicia = datos.wtseries.sel(lon=lon_galicia, lat=lat_galicia, method='nearest') 
#Este archivo es un xarray.core.dataarray.DataArray



periodo=pd.date_range(start=datos_tiempo[0], end=datos_tiempo[-1], freq="6h")
#El formato es pandas.core.indexes.datetimes.DatetimeIndex

#### En esta parte filtramos la mitad del año de verano

In [5]:
meses=[10,11,12,1,2,3]
labelmeses='ONDEFM'

locmeses=np.where(periodo.month.isin(meses))[0] #El formato es un array

datos_galicia=datos_galicia.isel(time=locmeses)
periodo=periodo[locmeses] #El formato es pandas.core.indexes.datetimes.DatetimeIndex

#### Defino la función de regresión lineal

In [6]:
def func(x, a, b):
    y=a*x+b
    return y


#### Sacamos, para cada clase, la frecuencia con respecto al año y lo guardamos en una matriz

In [7]:
years= np.unique(periodo.year) #El formato es array
freq_año=np.zeros((27, len(years)))
for i in years:
  locaño=np.where(periodo.year.values==i)[0]
  datos_galicia_año=datos_galicia.isel(time=locaño) #El formato es xarray.core.dataarray.DataArray
  freq_año[:, i-1979]=np.histogram(datos_galicia_año.values, bins=np.arange(1,29))[0]
  

#### En esta parte sacamos la frecuencia relativa de cada clase creando una tabla de pandas para la misma, representamos en grafico de barras la frecuencia absoluta con respecto a los años de cada clase, representamos la recta obtenida de una regresión lineal de dichos datos sobre la anterior representación y guardamos cada figura. Finalmente hacemos un Mann Kendall Trend Test para sacar el p-valor de dicha tendencia lineal

In [8]:
freq_relativa=[]
for i in range(0, 27):
    x=years
    y=freq_año[i,:]
    popt, pcov = curve_fit(func, x, y)
    a, b=popt
    MannKendall = mk.original_test(y)
    freq_total=np.sum(y)
    freq_rel=freq_total/len(datos_galicia.time.values)
    freq_relativa.append(freq_rel)
    print('Clase '+str(datos_tipos[i]))
    print('a =',a)
    print('b =',b)
    print('Test MannKendall:', MannKendall)
    figure=plt.figure()
    plt.bar(x, y)
    plt.plot(x, func(x,a,b), color='green')
    plt.xlabel('Año')
    plt.ylabel('Frecuencia')
    plt.title(labelregion+' Clase:'+str(datos_tipos[i]) +'  '+ labelmeses + '  Slope:'+str(np.round(a, decimals=4))+'  p-valor:'+str(np.round(MannKendall[2], decimals=4)))
    nombre_figure=labelregion+'_'+labelmeses+'_'+'_frecuencia_'+str(datos_tipos[i])+'.jpg'
    plt.savefig(nombre_figure)
    plt.close()
    print('=================')
d = {'Clase': datos_tipos, 'Frecuencia relativa': list(freq_relativa)}
tabla_freq_rel=pd.DataFrame(data=d)

Clase PA
a = -0.4626853512247215
b = 1114.4726931071639
Test MannKendall: Mann_Kendall_Test(trend='no trend', h=False, p=0.2596491005605346, z=-1.1272208893864075, Tau=-0.12195121951219512, s=-105.0, var_s=8512.333333333334, slope=-0.5714285714285714, intercept=197.21428571428572)
Clase DANE
a = 0.2526537557718126
b = -489.3716608605444
Test MannKendall: Mann_Kendall_Test(trend='increasing', h=True, p=0.009440386860767536, z=2.595680262930114, Tau=0.2787456445993031, s=240.0, var_s=8478.0, slope=0.2222222222222222, intercept=9.444444444444445)
Clase DAE
a = 0.14042622153608808
b = -264.85365853716536
Test MannKendall: Mann_Kendall_Test(trend='no trend', h=False, p=0.2775757895192772, z=1.085781236901792, Tau=0.1173054587688734, s=101.0, var_s=8482.333333333334, slope=0.13636363636363635, intercept=10.704545454545455)
Clase DASE
a = -0.06620209059466342
b = 140.2044134730167
Test MannKendall: Mann_Kendall_Test(trend='no trend', h=False, p=0.28610290994150844, z=-1.0667097788899509, Tau=

In [9]:
print(tabla_freq_rel)
datos.close() 
print('script acabo')

   Clase  Frecuencia relativa
0     PA             0.259700
1   DANE             0.021685
2    DAE             0.021848
3   DASE             0.010745
4    DAS             0.010516
5   DASW             0.039941
6    DAW             0.047649
7   DANW             0.027727
8    DAN             0.017211
9   PDNE             0.037067
10   PDE             0.046897
11  PDSE             0.026192
12   PDS             0.028805
13  PDSW             0.084912
14   PDW             0.083769
15  PDNW             0.052090
16   PDN             0.033671
17    PC             0.056956
18  DCNE             0.006695
19   DCE             0.006140
20  DCSE             0.007250
21   DCS             0.007969
22  DCSW             0.013227
23   DCW             0.008981
24  DCNW             0.006564
25   DCN             0.006532
26     U             0.029262
script acabo
